# <font color=#B2D732> <span style="background-color: #4424D6"> Brain & Spinal Cord fMRI Quality check </font>
<hr style="border:1px solid black">  

*Project: 2024_brsc_aging_project*  
*Paper: *  
**@ author:** 
> Caroline Landelle, caroline.landelle@mcgill.ca // landelle.caroline@gmail.com   
> June 2024 (last update: 4 June 2024)

**Description:** 
> This notebook provides code for quality check of fMRI data of simultaneous brain and spinal cord acquisition (single FOV).   


**Associated files:**
> - ../config/02_brsc_preprocess.json ; This file contains the directories and should be modified first
> - ../.py ; Nothing should be changed in this file
> - ../.py; Nothing should be changed in this file

**Steps:**
> **I. Participants **  

> **II. Framewise displacement**  


> **III. tSNR**  


<hr style="border:1px solid black">


## <font color=#B2D732> <span style="background-color: #4424D6">  Imports & configuration </font >

In [ ]:
import sys, json, glob, os
import pandas as pd
import numpy as np

main_dir="/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/"
code_dir=main_dir + "/2025_brsc_aging_project/"
os.chdir(code_dir)
sys.path.append(code_dir + "/code/") # Change this line according to your directory
# Load config file ------------------------------------------------------------
# The config file should be first review and all path should be modified (lines: )
with open(code_dir + '/config/preprocessing/01_brsc_preprocess_func.json') as config_file: # the notebook should be in 'xx/notebook/' folder 
    config = json.load(config_file) # load config file should be open first and the path inside modified

import brsc_utils as utils
import brsc_QC as QC
from brsc_preprocess import Preprocess_BrSc, Preprocess_Sc, Preprocess_Br

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
from plotting import Plotting
plot=Plotting(config_file.name,"test")

#statistics
from brsc_statistics import Statistics 
stat_func=Statistics(config=config,IDs=config["participants_IDs_dartel"],ana_dir="",analysis="")
import statsmodels.api as sm

%load_ext autoreload
%autoreload 2
%matplotlib inline
    

#Load participant file
info_IDs=pd.read_csv(code_dir +'/config/' +config["design_exp"]["info_participants"],delimiter="\t")

<hr style="border:1px solid black">    

## <font color=#B2D732> <span style="background-color: #4424D6"> I. Participants  </font>

In [ ]:
# Calculate mean and standard deviation of age
print("Mean FD brain: "+ str(np.round(info_IDs['age'].mean(),2)) + " ± "+ str(np.round(info_IDs['age'].std(),2)) + " mm")
    

print(" ")
# Count the number of male and female
sex_counts = info_IDs['sex'].value_counts()
print("Number of male (M):", sex_counts['M'], "and Number of female (F):", sex_counts['F'])

for modality in ["fmri","T1w","T2w","MTR","DWI"]:
    print(" ")
    print("----------------------")
    print("For " + modality)
    sub_df=info_IDs[info_IDs["exclusion_" + modality] ==0]
    print("Mean : "+ str(np.round(sub_df['age'].mean(),2)) + " ± "+ str(np.round(sub_df['age'].std(),2)))
    sex_counts = sub_df['sex'].value_counts()
    print("Number of male (M):", sex_counts['M'] , "and Number of female (F):", sex_counts['F'])
    print()

print(" ")
print("----------------------")
sub_df=info_IDs[(info_IDs["exclusion_MTR"] ==0) & (info_IDs["exclusion_DWI"] ==0) ]
print("Mean FD brain: "+ str(np.round(sub_df['age'].mean(),2)) + " ± "+ str(np.round(sub_df['age'].std(),2)) + " mm")
sex_counts = sub_df['sex'].value_counts()
print("Number of male (M):", sex_counts['M'] , "and Number of female (F):", sex_counts['F'])
  

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
colors = ["#0C675F","#3AA198","#68BCB4","#86C9C3","#C3E4E1","#F1C2CF","#E3849F","#D5476F","#C7093F","#850429"]  # blue → white → red
custom_cmap = LinearSegmentedColormap.from_list("my_colormap", colors)

output_dir=main_dir + "/figures/fS01_QC/"
plot.boxplots(df=info_IDs,
              x_data="sex",x_order=["F","M"],
                  indiv_values=True,#invers_axes=True,
              indiv_hue="age",indiv_color=custom_cmap,
              palette=["gray","gray"],
              height=3,aspect=0.5,
                  y_data="age",
              ymin=10, ymax=90,
              output_dir=output_dir ,
              output_tag="age_by_sex",plot_legend=False,
              save=False)
plt.show()

#info_IDs.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_S2A_QC_population.csv", index=False)

In [ ]:
output_dir=config["project_dir"] +  "/figures/fS01_QC/revision_R1/"
# Assuming your DataFrame is called df
#sns.set(style="whitegrid")

plt.figure(figsize=(8,5))
sns.histplot(data=info_IDs, x="age", bins=10, color="gray")
plt.title("Histogram of Age by Sex")
plt.xlabel("Age")
plt.ylabel("Count")
#plt.show()
plt.savefig(output_dir + "age_by_sex_histogram.pdf", dpi=300, bbox_inches="tight")


<hr style="border:1px solid black">    

## <font color=#B2D732> <span style="background-color: #4424D6"> II. Framewise displacement  </font>

In [ ]:
IDs=[];FDmean_sc_x=[];FDmean_sc_y=[];FDmean_spinalcord=[];FDmean_brain=[]; df={}
ID_file=config["tools_dir"]["main_codes"] + "/config/" + config["design_exp"]["info_participants"]
included_df=info_IDs[info_IDs["exclusion_fmri"]==0]

for structure in ["brain","spinalcord"]:#config["structures"]:
    for ID_nb,ID in enumerate(included_df["participant_id"]):

        directory="main_dir" if ID[0]=="A" else "bmpd_dir"
        ID_preproc_dir=config["main_dir"]+config["preprocess_dir"][directory] + "/sub-" + ID
        if structure=="spinalcord":
            FDmean_sc_x.append(pd.read_csv(ID_preproc_dir +"/func/"+config["preprocess_dir"]["func_moco"]   +"/spinalcord/FD_mean.txt" ,delimiter=" ",header=None)[0][0])
            FDmean_sc_y.append(pd.read_csv(ID_preproc_dir +"/func/"+config["preprocess_dir"]["func_moco"]   +"/spinalcord/FD_mean.txt" ,delimiter=" ",header=None)[1][0])
            FDmean_spinalcord.append(np.mean([FDmean_sc_y[ID_nb],FDmean_sc_x[ID_nb],FDmean_sc_y[ID_nb]]))
        else:
            FDmean_brain.append(pd.read_csv(ID_preproc_dir+"/func/"+config["preprocess_dir"]["func_moco"]   +"/brain/FD_mean.txt" ,delimiter=" ",header=None)[0][0])
    
    variable_name = f"FDmean_{structure}"
    df[structure] = pd.DataFrame({'ID': included_df["participant_id"],'group': included_df["group"],'age': included_df["age"],'exclusion': included_df["exclusion_fmri"], 'FD': globals()[variable_name],'structure':  structure}).reset_index()

# Concatenate horizontally the two dataframes
df_FD= pd.concat([df["spinalcord"], df["brain"]])
df_FD.reset_index(drop=True, inplace=True)


# Calculate mean and standard deviation of FD for 
FD_sc_mean = np.nanmean(df["spinalcord"]['FD'])
FD_sc_std = np.nanmean(df["spinalcord"]['FD'])
print("Mean FD spinal cord: "+ str(np.round(FD_sc_mean,2)) + " ± "+ str(np.round(FD_sc_std,2)) + " mm")

# Calculate mean and standard deviation of FD for 
FD_brain_mean = np.nanmean(df["brain"]['FD'])
FD_brain_std = np.nanmean(df["brain"]['FD'])
print("Mean FD brain: "+ str(np.round(FD_brain_mean,2)) + " ± "+ str(np.round(FD_brain_std,2)) + " mm")

print("")
print("Exlcusion criteria: FD> mean ± 2 std")
for structure in ["spinalcord","brain"]:#config["structures"]:
    for ID_nb,ID in enumerate(included_df["participant_id"]):
        FD_mean=FD_sc_mean if structure=="spinalcord" else FD_brain_mean
        FD_std=FD_sc_std if structure=="spinalcord" else FD_brain_std
        
        if df[structure]['FD'][df[structure]["ID"]==ID][ID_nb]>(FD_mean + 2*FD_std):
            print(ID + ", "+structure+ " FD= "+str(np.round(df[structure]['FD'][df[structure]["ID"]==ID].reset_index()["FD"][0],3)) + "mm")
            df_FD.loc[df_FD["ID"]==ID,"exclusion"] =1

excl_FD=df_FD
incl_FD=df_FD[df_FD["exclusion"] <1].reset_index()

#hue='group',columns='structure',y_title="Framewise displacement (mm) ",save_plot=False)



In [ ]:

plot.boxplots(df=incl_FD,
              x_data="structure",
                  indiv_values=True,#invers_axes=True,
              x_order=["spinalcord","brain"],
              indiv_hue="age",indiv_color=custom_cmap,
              palette=["gray","gray"],#output_dir=config['main_dir'] + config['analysis_dir']['spinalcord'] + '/figures/',        
              height=3,aspect=0.55,
            y_data="FD",
              ymin=0, ymax=0.3,
              output_dir=output_dir ,
              output_tag="FD_by_structure",plot_legend=False,
              save=False)
plt.show()
#incl_FD.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_S2B_QC_FD.csv", index=False)

<hr style="border:1px solid black">    

## <font color=#B2D732> <span style="background-color: #4424D6"> III. tSNR  </font>

In [ ]:
tSNR_mean_spinalcord=[];tSNR_img_spinalcord=[];tSNR_template_spinalcord=[]; tSNR_mean_brain=[];tSNR_img_brain=[];tSNR_template_brain=[];df={}
info_IDs_incl=incl_FD[incl_FD["exclusion"]<1]
for structure in ["brain","spinalcord"]:
    print(structure)
    for ID_nb,ID in enumerate(np.unique(incl_FD["ID"])):
        if structure=="spinalcord":
            o_txt, native_tSNR, outreg_f=QC.tSNR(config=config,ID=ID,structure=structure,inTemplate = True) # to create the image
            tSNR_template_spinalcord.append(outreg_f)
            tSNR_mean_spinalcord.append(pd.read_csv(QC.tSNR(config=config,ID=ID,structure=structure)[0],delimiter=" ",header=None)[0][0])
            #tSNR_img_spinalcord.append(QC.tSNR(config=config,ID=ID,structure=structure,inTemplate = False)[2])
            
            
        elif structure=="brain":
            o_txt, native_tSNR, outreg_f=QC.tSNR(config=config,ID=ID,structure=structure,inTemplate = True, redo=False) # to create the image
            tSNR_template_brain.append(outreg_f)
            tSNR_mean_brain.append(pd.read_csv(QC.tSNR(config=config,ID=ID,structure=structure)[0],delimiter=" ",header=None)[1][0])
            #tSNR_img_brain.append(QC.tSNR(config=config,ID=ID,structure=structure,inTemplate = False,redo=False)[2])
        

    
    variable_name = f"tSNR_mean_{structure}"
    df[structure] = pd.DataFrame({'ID': info_IDs_incl["ID"][info_IDs_incl["structure"]=="spinalcord"],'age': info_IDs_incl["age"][info_IDs_incl["structure"]=="spinalcord"],'group': info_IDs_incl["group"][info_IDs_incl["structure"]=="spinalcord"],'tSNR': globals()[variable_name],'structure':  structure}).reset_index()

#Create a group image
QC.tSNR_group(config=config,i_img=tSNR_template_brain,structure="brain")
QC.tSNR_group(config=config,i_img=tSNR_template_spinalcord,structure="spinalcord")

# Concatenate horizontally the two dataframes
df_tSNR= pd.concat([df["spinalcord"], df["brain"]])
df_tSNR.reset_index(drop=True, inplace=True)


# Calculate mean and standard deviation of FD for 
tSNR_sc_mean = df["spinalcord"]['tSNR'].mean()
tSNR_sc_std = df["spinalcord"]['tSNR'].std()
print("Mean tSNR spinal cord:", tSNR_sc_mean)
print("Standard deviation :", tSNR_sc_std)

# Calculate mean and standard deviation of FD for 
tSNR_brain_mean = df["brain"]['tSNR'].mean()
tSNR_brain_std = df["brain"]['tSNR'].std()
print("Mean FD brain:", tSNR_brain_mean)
print("Standard deviation :", tSNR_brain_std)




In [ ]:
QC.tSNR_group(config=config,i_img=tSNR_template_brain,structure="brain")


In [ ]:
plot.boxplots(df=df_tSNR,
              x_data="structure",
                  indiv_values=True,#invers_axes=True,
                  indiv_hue="age",indiv_color=custom_cmap,
              palette=["gray","gray"],#output_dir=config['main_dir'] + config['analysis_dir']['spinalcord'] + '/figures/',        
                  #output_tag='corr_' + group_name,
              height=3,aspect=0.55,
                  y_data="tSNR",
              ymin=10, ymax=80,
              output_dir=output_dir ,
              output_tag="tSNR_by_structure",plot_legend=False,
              save=False)
plt.show()

#df_tSNR.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_S2C_QC_tSNR.csv", index=False)

#### Age regression and statistics

In [ ]:
#g=sns.lmplot(data=incl_FD, x="age", y="FD", hue="structure",palette=["#B14263","#CB8499"])
plot.lmplots(df=incl_FD,color=["#B5B5B5","#696666"],
                 x_data="age",
                 y_data="FD",hue_var="structure",#xmin=15,xmax=85,
             indiv_values=False,
             xmin=15,xmax=85,
                 ymin=0, ymax=0.15,xy_plot=True,
                  output_dir=output_dir,
                 output_tag="FD_by_structure_by_age",
                 save=False)

for structure in ["brain","spinalcord"]:
    print("Age regression: " + structure)
    sub_df=incl_FD[incl_FD["structure"]==structure]
    X= sm.add_constant(sub_df["age"])
    model_FD= sm.OLS(sub_df["FD"], X).fit()
    print(model_FD.summary())
    print("")

#incl_FD.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_S2B_QC_FD_aging.csv", index=False)

In [ ]:
#g=sns.lmplot(data=incl_FD, x="age", y="FD", hue="structure",palette=["#B14263","#CB8499"])
plot.lmplots(df=df_tSNR,color=["#B5B5B5","#696666"],
                 x_data="age",
                 y_data="tSNR",hue_var="structure",#xmin=15,xmax=85,
             indiv_values=False,
             xmin=15,xmax=85,
                 #ymin=0, ymax=0.15,
             xy_plot=True,
                  output_dir=output_dir,
                 output_tag="tSNR_by_structure_by_age",
                 save=False)

for structure in ["brain","spinalcord"]:
    print("Age regression: " + structure)
    sub_df=df_tSNR[df_tSNR["structure"]==structure]
    X= sm.add_constant(sub_df["age"])
    model_FD= sm.OLS(sub_df["tSNR"], X).fit()
    print(model_FD.summary())
    print("")
#df_tSNR.to_csv(config["project_dir"] +  "/figures/source_datafile/fig_S2C_QC_tSNR_aging.csv", index=False)        